In [1]:
import json
import http.client
from urllib.parse import urlencode, quote_plus, quote
import pandas as pd
from datetime import datetime
from tslearn.clustering import TimeSeriesKMeans, silhouette_score
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
from tslearn.utils import to_time_series_dataset

/home/clem/Documents/source/sandbox/webstat-tsclustering/.venv/lib/python3.8/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


# Import final dataframe

In [6]:
final_df = pd.read_csv("final_df.csv", index_col=0)
final_df_wona = final_df.dropna(axis=1)
normalized_final_df = (final_df_wona - final_df_wona.mean()) / final_df_wona.std()

# Convert to `tslearn` dataset

In [7]:

X_train = to_time_series_dataset(normalized_final_df.T)

In [8]:
X_train.shape # n_ts, max_sz, d

(517, 240, 1)

In [9]:
for cluster_count in range(2, 7):
    km = TimeSeriesKMeans(n_clusters=cluster_count, metric="dtw", n_jobs=2, n_init=5)
    labels = km.fit_predict(X_train)
    score = silhouette_score(X_train, labels, metric="dtw", n_jobs=4)
    print(f"{cluster_count}: {score}")


2: 0.34938194769965675
3: 0.26612659244609793
4: 0.25588602064105825
5: 0.27682811418614217
6: 0.2379344426483915


In [ ]:
CLUSTER_COUNT = 6
# km = TimeSeriesKMeans(n_clusters=CLUSTER_COUNT, metric="dtw", n_jobs=2, n_init=5)
# labels = km.fit_predict(X_train)

plt.figure(figsize=(18,14))

for yi in range(CLUSTER_COUNT):
    plt.subplot(CLUSTER_COUNT, 1, yi + 1)
    for xx in X_train[labels == yi]:
        plt.plot(xx.ravel(), "k-", alpha=0.2)
    plt.plot(km.cluster_centers_[yi].ravel(), "r-")
    # plt.xlim(0, 1000)
    # plt.ylim(-4, 4)
    plt.text(0.55, 0.85, "Cluster %d" % (yi + 1), transform=plt.gca().transAxes)


In [ ]:
# { i: column_name for i, column_name in enumerate(normalized_final_df.columns) }
# cluster_serie_indices = np.where(labels == i)[0]

column_names = np.array(normalized_final_df.columns)

cluster_sizes = []
names_by_cluster = []
series_by_cluster = []

for i in range(CLUSTER_COUNT):
    cluster_series = X_train[labels == i]
    series_by_cluster.append(cluster_series)

    cluster_serie_names = column_names[labels == i]
    names_by_cluster.append(cluster_serie_names)

    cluster_size = len(cluster_series)
    cluster_sizes.append(cluster_size)


In [ ]:
names_by_cluster[2]